In [27]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import pickle

def developer(desarrollador):
    '''
    Esta función devuelve información sobre una empresa desarrolladora de videojuegos.
         
    Args:
        desarrollador (str): Nombre del desarrollador de videojuegos.
    
    Returns:
        dict: Un diccionario que contiene información sobre la empresa desarrolladora.
            - 'cantidad_por_año' (dict): Cantidad de items desarrollados por año.
            - 'porcentaje_gratis_por_año' (dict): Porcentaje de contenido gratuito por año según la empresa desarrolladora.
    '''
    df_items_developer = pd.read_parquet('arch_parquet/df_items_developer.parquet')
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('año_lanzamiento')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('año_lanzamiento')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int) +'%'

    result_dict = {
        'cantidad_items_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict() 
    }
    
    return result_dict

def userdata(user_id):
    '''
    Esta función devuelve información sobre un usuario según su 'user_id'.
         
    Args:
        user_id (str): Identificador único del usuario.
    
    Returns:
        dict: Un diccionario que contiene información sobre el usuario.
            - 'cantidad_dinero' (int): Cantidad de dinero gastado por el usuario.
            - 'porcentaje_recomendacion' (float): Porcentaje de recomendaciones realizadas por el usuario.
            - 'total_items' (int): Cantidad de items que tiene el usuario.
    '''
    df_reviews = pd.read_parquet('arch_parquet/df_reviews.parquet')
    df_gastos_items = pd.read_parquet('arch_parquet/gastos_items.parquet')
    # Filtra por el usuario de interés
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['user_id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews['user_id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100

    
    return {
        'Dinero Gastado': str(int(cantidad_dinero)) + ' USD',
        '% de recomendación': str(round(float(porcentaje_recomendaciones), 2)*100) +' %',
        'Cantidad de items': int(count_items)
    }


def get_playtime_by_genre2(genero: str):
    df_año_horas = pd.read_parquet('arch_parquet\df_anio_horas.parquet')
    df_año_horas = df_año_horas[['genres','año_lanzamiento','playtime_horas']]
    df_filtrado = df_año_horas[df_año_horas['genres'] == genero]
    if df_filtrado.empty:
        return {"message": f"No data found for genre: {genero}"}
    playtime_sum = df_filtrado.groupby('año_lanzamiento')['playtime_horas'].sum() 
    año_max_jugado = playtime_sum.idxmax()
    max_playtime = playtime_sum.max()
    del df_año_horas, df_filtrado, playtime_sum
    return {"Año de lanzamiento con más horas jugadas para el género: " + genero : str(año_max_jugado), "Horas jugadas": str(max_playtime)}

def UserForGenre(genero: str):
    try:
        df_año_horas = pd.read_parquet('arch_parquet\df_anio_horas.parquet')
        df_año_horas = df_año_horas[['genres','user_id', 'año_lanzamiento','playtime_horas']]
        df_filtrado = df_año_horas[df_año_horas['genres'] == genero]
        playtime_sum = df_filtrado.groupby(['user_id', 'año_lanzamiento'])['playtime_horas'].sum() 
        user_max_playtime = playtime_sum.groupby('user_id').sum().idxmax()
        playtime_by_year = playtime_sum.loc[user_max_playtime].to_dict()
        del df_año_horas, df_filtrado, playtime_sum
        return {"Usuario con más horas jugadas para el género: " + genero : user_max_playtime, "Horas jugadas": playtime_by_year}
    except Exception as e:
        return {"error": str(e)}

def get_users_recommend(año: int):
    review_games = pd.read_parquet('arch_parquet\df_reviews.parquet')
    review_games = review_games[['developer','año_lanzamiento','title', 'reviews_recommend','sentiment_analysis']]
    df_filtered = review_games[(review_games['año_lanzamiento'] == año) & (review_games['reviews_recommend'] == True) & (review_games['sentiment_analysis'] >= 1)]
    recommend_count = df_filtered['title'].value_counts()
    top_3_games = recommend_count.nlargest(3).index.tolist()
    del review_games, df_filtered, recommend_count
    return [{"Puesto 1" : top_3_games[0]}, {"Puesto 2" : top_3_games[1]}, {"Puesto 3" : top_3_games[2]}]

def get_best_developer(año: int):
    review_games = pd.read_parquet('arch_parquet\df_reviews.parquet')
    review_games = review_games[['developer','año_lanzamiento','reviews_recommend', 'sentiment_analysis']]
    df_filtered = review_games[(review_games['año_lanzamiento'] == año) & (review_games['reviews_recommend'] == True) & (review_games['sentiment_analysis'] >= 1)]
    positive_reviws_count = df_filtered['developer'].value_counts()
    top_3_best_developers = positive_reviws_count.nlargest(3).index.tolist()
    del review_games, df_filtered, positive_reviws_count
    return [{"Puesto 1" : top_3_best_developers[0]}, {"Puesto 2" : top_3_best_developers[1]}, {"Puesto 3" : top_3_best_developers[2]}]

def get_worst_developer(año: int):
    review_games = pd.read_parquet('arch_parquet\df_reviews.parquet')
    review_games = review_games[['developer','año_lanzamiento','reviews_recommend', 'sentiment_analysis']]
    df_filtered = review_games[(review_games['año_lanzamiento'] == año) & (review_games['reviews_recommend'] == False) & (review_games['sentiment_analysis'] == 0)]
    negative_reviews_count = df_filtered['developer'].value_counts()
    top_3_bad_developers = negative_reviews_count.nlargest(3).index.tolist()
    del review_games, df_filtered, negative_reviews_count
    return [{"Puesto 1" : top_3_bad_developers[0]}, {"Puesto 2" : top_3_bad_developers[1]}, {"Puesto 3" : top_3_bad_developers[2]}]

def developer_reviews_analysis(developer:str):
    """
    Esta funcion calcula para un desarrolador la cantidad de usuarios con reviews positivas y negativas.
    params:
    developer_rec:str : Desarrolador
    """
    # Carga los datos de los juegos de steam
    df_games = pd.read_csv('Datasets_limpio/steam_games.csv')
    # Tomo solo un 10% de mi df:
    df_games= df_games.sample(frac=0.1,random_state=42)
    # Carga las revisiones de los usuarios
    df_reviews = pd.read_csv('Datasets_limpio/df_reviews.csv')
    # Tomo solo un 10% de mi df:
    #df_reviews= df_reviews.sample(frac=0.1,random_state=42)
    df_games['id'] = df_games['id'].astype(int)
    df_reviews['reviews_item_id'] = df_reviews['reviews_item_id'].astype(int)
    # Merging los dos datasets, con una combinación interna en sus respectivos 'id'
    func_5 = pd.merge(df_reviews,df_games,left_on='reviews_item_id',right_on='id',how='inner')
    # Convertir todos los nombres de los desarrolladores en letras minúsculas para evitar la duplicación de datos debido a las diferencias de mayúsculas y minúsculas
    func_5['developer_x'] = func_5['developer_x'].str.lower()
    # Convertir el nombre del desarrollador proporcionado en letras minúsculas
    developer2 = developer.lower()
    # Filtrar por desarrollador
    func_5 = func_5[func_5['developer_x'] == developer2]
    # Verificar si se encuentra los juegos del desarrollador en el dataset
    if func_5.empty:
        # En caso de que no se encuentre, se muestra mensaje indicando que no hay comentarios para este desarrollador
        return 'No se encontraron reviews para ese desarrollador'
    # En caso contrario, contar los sentimientos de análisis de comentarios
    # Cuenta los comentarios positivos
    true_value = func_5[func_5['sentiment_analysis']==2]['sentiment_analysis'].count()
    # Cuenta los comentarios negativos
    false_value = func_5[func_5['sentiment_analysis']==0]['sentiment_analysis'].count()
    # Devolver conteos en un diccionario
    return {developer2:[f'Negative = {int(false_value)}',f'Positive = {int(true_value)}']}

def recomendacion_juego(game):

    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.
  
    Returns:
        None: Un diccionario con 5 nombres de juegos recomendados.

    '''
    item_sim_df = pd.read_parquet('archivos_parquet/item_sim_df.parquet')
    # Obtener la lista de juegos similares ordenados
    similar_games = item_sim_df.sort_values(by=game, ascending=False).iloc[1:6]

    count = 1
    contador = 1
    recomendaciones = {}
    
    for item in similar_games:
        if contador <= 5:
            item = str(item)
            recomendaciones[count] = item
            count += 1
            contador += 1 
        else:
            break
    return recomendaciones


def get_recommendation(item_id: int):
    df_final = pd.read_parquet('arch_parquet\df_reviews.parquet')
    df_final = df_final[['reviews_item_id', 'user_id', 'reviews_recommend','title']] 
    try:
        with open('indices_similares.pkl', 'rb') as f:
            indices_similares = pickle.load(f)
    except FileNotFoundError:
        sparse_matrix = csr_matrix(pd.crosstab(df_final['reviews_item_id'], df_final['user_id'], values=df_final['reviews_recommend'], aggfunc='sum').fillna(0))
        similitud = cosine_similarity(sparse_matrix)
        indices_similares = {item_id: similitud[item_index].argsort()[-6:-1][::-1] for item_index, item_id in enumerate(df_final['reviews_item_id'].unique().tolist())}
        with open('indices_similares.pkl', 'wb') as f:
            pickle.dump(indices_similares, f)
        del sparse_matrix, similitud
    similar_items = [df_final['title'].unique().tolist()[i] for i in indices_similares[item_id]]
    del df_final
    return similar_items

In [28]:
recomendacion_juego(643980)

FileNotFoundError: [Errno 2] No such file or directory: 'archivos_parquet/item_sim_df.parquet'

In [26]:
developer_reviews_analysis('Valve')

{'valve': ['Negative = 588', 'Positive = 2559']}

In [4]:
userdata('Darkjet15')

{'Dinero Gastado': '978 USD',
 '% de recomendación': '3.0 %',
 'Cantidad de items': 119}

In [5]:
UserForGenre('Action')

{'Usuario con más horas jugadas para el género: Action': 'Sp3ctre',
 'Horas jugadas': {0: 109,
  1993: 0,
  1995: 3,
  1996: 0,
  1998: 0,
  1999: 0,
  2000: 1177,
  2001: 0,
  2002: 3,
  2003: 127,
  2004: 2122,
  2005: 355,
  2006: 13,
  2007: 1878,
  2008: 3,
  2009: 1803,
  2010: 1297,
  2011: 2574,
  2012: 6291,
  2013: 1998,
  2014: 2165,
  2015: 5110,
  2016: 488,
  2017: 720}}

In [17]:
developer('Valve')

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int32'), dtype('<U1')) -> None

In [ ]:
get_worst_developer(2012)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Wild Shadow Studios'},
 {'Puesto 3': 'Jagex Limited'}]

In [ ]:
get_best_developer(2012)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Gearbox Software,Aspyr (Mac &amp; Linux)'},
 {'Puesto 3': 'Daybreak Game Company'}]

In [ ]:
get_users_recommend(2012)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': 'Borderlands 2'},
 {'Puesto 3': 'PlanetSide 2'}]